In [ ]:
# enviroment: flappy bird
    # 4 states (normalized), 2 actions
    #  reward: -5 die, +0.05 survive, +5 pass pipe

# framework: tf.keras
# model: RL PPO (2 NN)
    # method: Advantage = Q - value where Q = R + discount * Q'
    # Actor: 4 input | 64 relu | 64 relu | softmax -> policy. loss: mean (log policy * advantage)
    # Critic: 4 input | 64 relu | 64 relu | linear -> value. loss: mean square (advantage)
    # params: discount 0.999, optimizer adam-learning rate 0.001, no e-greedy

# result:

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
from enviroment import FlappyBird
from modelPPO import PPO